In [ ]:
from huggingface_hub import InferenceClient, login
import os
import json
import time
from datetime import datetime, timedelta
login(os.getenv('HF_TOKEN')) # need huggingface token generated from site for account to use certain models.

In [4]:
models = [
    'mistralai/Mistral-7B-Instruct-v0.3',
    'meta-llama/Meta-Llama-3-8B-Instruct',
    "microsoft/Phi-3.5-mini-instruct",
    # "HuggingFaceH4/zephyr-7b-beta"
    # 'google/gemma-2-9b-it' # Doesn't work with hf_hub==0.24.5 (don't want to remove system message functionality)
]

In [5]:
def let_llm_judge_name(name, sql, model):

    client = InferenceClient(model)
    messages = [
    {"role": "system",
     "content": "You are a helpful SQL assistant evaluating names to common table expressions. The names should reflect what result set the SQL is trying to produce"},
    {"role": "user",
     "content": f"Consider the following SQL snippet\n```sql\n{sql}```\n\nEvaluate this name for the SQL statement as a common table expression from 1 to 10, 1 being the least comprehensive, and 10 being the most, return only the nummeric value from 1 to 10 and nothing else:\n{name}"},
    ]
    model_response = client.chat.completions.create(messages=messages, max_tokens=2000, seed=42, temperature=0)
    return model_response.choices[0].message.content


def write_in_llm_evaluations(json_cte_data_file):
    with open(json_cte_data_file, 'r+') as file:
        cte_data = json.load(file)

        for i, obj in enumerate(cte_data):
            for model in models:
                if f"{model.split('/')[1]}-name-evaluation" not in obj or not obj[f"{model.split('/')[1]}-name-evaluation"]:
                    try:
                        obj[f"{model.split('/')[1]}-name-evaluation" ] = let_llm_judge_name(obj["cte-name"], obj['SQL'], model)
                        file.seek(0)
                        json.dump(cte_data, file, indent=2)
                        file.truncate()
                    except Exception as e:
                        print(f"Error generating summary for {json_cte_data_file}; model {model} at index {i}: {e}")
                        # Pause and return to allow rerun later
                        return -1
    return 0

def sleep_until_next_hour():
    now = datetime.now()
    # Calculate the time until the next hour
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
    sleep_time = (next_hour - now).total_seconds()
    print(f"Sleeping for {int(sleep_time // 60)} minutes until {next_hour}.")
    time.sleep(sleep_time)


In [ ]:
json_data = 'data_files/spider2-lite-ctes.json'

result = -1
while result != 0:
    result = write_in_llm_evaluations(json_data)
    if result == -1:
        sleep_until_next_hour()
    elif result == 0:
        print("Finished successfully.")
        break


#### Use informed prompts and personas to judge things

In [6]:
import re
def extract_first_numeric_value(text):
    for token in re.split(r'\s+', text):
        if token.isnumeric():
            if int(token) in range(0,11):
                return int(token)
    return None


def let_llm_judge_name_as_persona(name, sql, model):

    cte_eval_prompt = f"""
        You will evaluate the comprehensiveness of the name assigned to this common table expression (CTE) from 1 to 10, where:
        - 1 indicates a name that is vague or unrelated, and
        - 10 indicates a highly descriptive, accurate name that clearly reflects the CTE's purpose.

        Examples:

        1. SQL: SELECT customer_id, SUM(order_total) AS total_spent FROM orders GROUP BY customer_id
            - Name: total_spent_per_customer – Expected score: 9
            - Name: customer_data – Expected score: 3
        2. SQL: SELECT product_id, COUNT(*) AS order_count FROM order_details GROUP BY product_id
            - Name: ProductOrderCounts – Expected score: 10
            - Name: details – Expected score: 2
        3. SQL: SELECT order_id, MAX(order_date) FROM orders GROUP BY order_id
            - Name: latest_order_dates – Expected score: 8
            - Name: order_info – Expected score: 4

        Placeholder names like tmp, aaa, cte1, table1, mycte, etc. are placeholder names that say nothing about the CTE, so they would recieve scores of 1-2.

        Assume the following SQL snippet:
        ```sql
        {sql}
        ```
        Your task: Evaluate this proposed name for the SQL statement in terms of its comprehensiveness. Provide only a single numeric score from 1 to 10 as output.

        Name to evaluate: {name}

        """
    

    client = InferenceClient(model)
    messages = [
    {"role": "system",
     "content": f"You are a graduate student in computer science specializing in databases and skilled in SQ, and you're tasked with evaluating names to common table expressions. The names should reflect what result set the SQL is trying to produce"},
    {"role": "user",
     "content": cte_eval_prompt},
    ]
    model_response = client.chat.completions.create(messages=messages, max_tokens=2000, seed=42, temperature=0)
    return model_response.choices[0].message.content


def write_in_llm_evaluations_both_files(json_cte_data_file):
    with open(json_cte_data_file, 'r+') as file:
        cte_data = json.load(file)

        for i, obj in cte_data:
            for model in models:
                if f"{model.split('/')[1]}-name-evaluation" not in obj or not obj[f"{model.split('/')[1]}-name-evaluation"] or extract_first_numeric_value(obj[f"{model.split('/')[1]}-name-evaluation"]) is None:
                    try:
                        obj[f"{model.split('/')[1]}-name-evaluation" ] = let_llm_judge_name_as_persona(obj["cte-name"], obj['SQL'], model)
                        file.seek(0)
                        json.dump(cte_data, file, indent=2)
                        file.truncate()
                    except Exception as e:
                        print(f"Error generating summary for {json_cte_data_file}; model {model} at index {i}: {e}")
                        # Pause and return to allow rerun later
                        return -1
    return 0

def sleep_until_next_hour():
    now = datetime.now()
    # Calculate the time until the next hour
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
    sleep_time = (next_hour - now).total_seconds()
    print(f"Sleeping for {int(sleep_time // 60)} minutes until {next_hour}.")
    time.sleep(sleep_time)


In [ ]:
student_data = 'data_files/spider2-lite-ctes.json'


result = -1
while result != 0:
    result = write_in_llm_evaluations_both_files(student_data, ds_data)
    if result == -1:
        sleep_until_next_hour()
    elif result == 0:
        print("Finished successfully.")
        break


#### Work on Curated Dataset as well

In [7]:
def write_in_llm_evaluations_both_files2(json_cte_data_file, json_cte_data_file_2):
    with open(json_cte_data_file, 'r+') as file, open(json_cte_data_file_2, 'r+') as file2:
        cte_data = json.load(file)
        cte_data_2 = json.load(file2)

        persona1 = "graduate student in computer science specializing in databases and skilled in SQL"
        persona2 = "data scientist who does a lot of data wrangling in SQL and uses temp tables and CTES"

        for i, (obj, obj2) in enumerate(zip(cte_data, cte_data_2)):
            for model in models:
                if f"{model.split('/')[1]}-name-evaluation" not in obj or not obj[f"{model.split('/')[1]}-name-evaluation"] or extract_first_numeric_value(obj[f"{model.split('/')[1]}-name-evaluation"]) is None:
                    try:
                        obj[f"{model.split('/')[1]}-name-evaluation" ] = let_llm_judge_name_as_persona(obj["cte_name"], obj['SQL'], model, persona1)
                        file.seek(0)
                        json.dump(cte_data, file, indent=2)
                        file.truncate()
                    except Exception as e:
                        print(f"Error generating summary for {json_cte_data_file}; model {model} at index {i}: {e}")
                        # Pause and return to allow rerun later
                        return -1
                if f"{model.split('/')[1]}-name-evaluation" not in obj2 or not obj2[f"{model.split('/')[1]}-name-evaluation"] or extract_first_numeric_value(obj2[f"{model.split('/')[1]}-name-evaluation"]) is None:
                    try:
                        obj2[f"{model.split('/')[1]}-name-evaluation" ] = let_llm_judge_name_as_persona(obj2["cte_name"], obj2['SQL'], model, persona2)
                        file2.seek(0)
                        json.dump(cte_data_2, file2, indent=2)
                        file2.truncate()
                    except Exception as e:
                        print(f"Error generating summary for {json_cte_data_file_2}; model {model} at index {i}: {e}")
                        # Pause and return to allow rerun later
                        return -1
    return 0

In [8]:
student_data = 'data_files/curated/curated_ctes_student_llm_judged.json'
ds_data = 'data_files/curated/curated_ctes_ds_llm_judged.json'


result = -1
while result != 0:
    result = write_in_llm_evaluations_both_files2(student_data, ds_data)
    if result == -1:
        sleep_until_next_hour()
    elif result == 0:
        print("Finished successfully.")
        break


Finished successfully.
